In [ ]:
#Importing required packages
from __future__ import print_function
import pandas as pd
import numpy as np
import re
import nltk
import sys
from keras.layers import Input, Bidirectional,Embedding, Dense, LSTM, TimeDistributed 
from keras.models import Model, load_model

Using TensorFlow backend.


In [ ]:
#Importing the movie_lines and movie_conversations data files
movie_lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
movie_conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [ ]:
# Mapping the line id with the text and storing in a dictionary
idx_2_line = {}
for conversation in movie_lines:
    _conversation = conversation.split(' +++$+++ ')
    if len(_conversation) == 5:
        idx_2_line[_conversation[0]] = _conversation[4]

In [ ]:
#list of all the conversations and its line id's
conversation_list = []
for conversation in movie_conv_lines[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversation_list.append(_conversation.split(','))

In [ ]:
#Categorizing the text into questions and answers and storing each in different lists
conversation_1 = []
conversation_2 = []
for dialogue in conversation_list:
    for i in range(len(dialogue)-1):
        conversation_1.append(idx_2_line[dialogue[i]])
        conversation_2.append(idx_2_line[dialogue[i+1]])
        
# Comparing the lengths of conversation_1 and conversation_2
print(len(conversation_1))
print(len(conversation_2))


221616
221616


In [ ]:
#Defining a function to clean the text 
def clean_sentence(line):
        line = line.lower()
        line = re.sub(r"i'm", "i am", line)
        line = re.sub(r"he's", "he is", line)
        line = re.sub(r"she's", "she is", line)
        line = re.sub(r"that's", "that is", line)
        line = re.sub(r"what's", "what is", line)
        line = re.sub(r"where's", "where is", line)
        line = re.sub(r"how's", "how is", line)
        line = re.sub(r"\'ll", " will", line)
        line = re.sub(r"\'ve", " have", line)
        line = re.sub(r"\'re", " are", line)
        line = re.sub(r"\'d", " would", line)
        line = re.sub(r"n't", " not", line)
        line = re.sub(r"won't", "will not", line)
        line = re.sub(r"can't", "cannot", line)
        line = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", line)
        line = " ".join(line.split())
        return line

In [ ]:
# Using the above function to Clean the text
clean_conversation_1 = []
for dialogue in conversation_1:
    clean_conversation_1.append(clean_sentence(dialogue))
clean_conversation_2 = []
for dialogue in conversation_2:
    clean_conversation_2.append(clean_sentence(dialogue))

In [ ]:
#Checking the length of the text in each utterance
conversation_length = []
for dialogue in clean_conversation_1:
    conversation_length.append(len(dialogue.split()))
for dialogue in clean_conversation_2:
    conversation_length.append(len(dialogue.split()))
conversation_length = pd.DataFrame(conversation_length, columns=['counts'])
print(np.percentile(conversation_length, 80))
print(np.percentile(conversation_length, 85))
print(np.percentile(conversation_length, 90))
print(np.percentile(conversation_length, 95))


16.0
19.0
24.0
32.0


In [ ]:
# Removing the sentences with less than 2 words or greater than 20 words.
conv_min_length = 2
conv_max_length = 20

conv_1_short_filt = []
conv_2_short_filt = []

for i, dialogue in enumerate(clean_conversation_1):
    if len(dialogue.split()) >= conv_min_length and len(dialogue.split()) <= conv_max_length:
        conv_1_short_filt.append(dialogue)
        conv_2_short_filt.append(clean_conversation_2[i])
        
conv_1_short = []
conv_2_short = []

for i, dialogue in enumerate(conv_2_short_filt):
    if len(dialogue.split()) >= conv_min_length and len(dialogue.split()) <= conv_max_length:
        conv_2_short.append(dialogue)
        conv_1_short.append(conv_1_short_filt[i])
        
print(len(conv_1_short))
print(len(conv_2_short))


138574
138574


In [ ]:
rand_no = np.random.randint(1,len(conv_1_short))
for i in range(rand_no, rand_no+3):
    print(conv_1_short[i])
    print(conv_2_short[i],'\n')

thank you baron
not at all countess there's one very good stone in it 

not at all countess there's one very good stone in it
what time is it 

darling now tell metell me all about yourself who are you
you remember the man who walked into the bank of constantinople and walked out uwithu the bank of constantinople 



In [ ]:
#Downloading punkt library 
import nltk
nltk.download()
#nltk.download(punkt)

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  d



Download which package (l=list; x=cancel)?


  Identifier>  punkt


      Unzipping tokenizers/punkt.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  q


True

In [ ]:
#Number of records to build the model
no_records = 100000
conv_1_short = conv_1_short[:no_records]
conv_2_short = conv_2_short[:no_records]
#Tokenizing the words
conv_1_short_nltk = [nltk.word_tokenize(dial) for dial in conv_1_short]
conv_2_short_nltk = [nltk.word_tokenize(dial) for dial in conv_2_short]

In [ ]:
#Splitting the data into train and Validation
split_len = len(conv_1_short_nltk)

#Using 80% of the data for training
data_train  = conv_1_short_nltk[:round(split_len*(80/100))]
data_train  = [record[::-1] for record in data_train]
data_outcome = conv_2_short_nltk[:round(split_len*(80/100))]

#20% of the data for Validation
data_valid_inp = conv_1_short_nltk[round(split_len*(80/100)):]
data_valid_inp  = [record[::-1] for record in data_valid_inp] 
validation_output = conv_2_short_nltk[round(split_len*(80/100)):]

print('training size', len(data_train))
print('validation size', len(data_valid_inp))

training size 80000
validation size 20000


In [ ]:
# data_vocabulary dictionary will store the freq of each word
data_vocabulary = {}
for dialogue in conv_1_short_nltk:
    for token in dialogue:
        if token not in data_vocabulary:
            data_vocabulary[token] = 1
        else:
            data_vocabulary[token] += 1

for dialogue in conv_2_short_nltk:
    for token in dialogue:
        if token not in data_vocabulary:
            data_vocabulary[token] = 1
        else:
            data_vocabulary[token] += 1

In [ ]:
#Removing the words that occur less than 5% of the time in the Vocabulary 
min_thres_value = 25
vocab_count = 0
for k,v in data_vocabulary.items():
    if v >= min_thres_value:
        vocab_count += 1

In [ ]:
print("Vocabulary size before removing the rare words:", len(data_vocabulary))
print("Vocab size after removing the rare words:", vocab_count)

Vocabulary size before removing the rare words: 35117
Vocab size after removing the rare words: 3168


In [ ]:
#Assigning a unique number to each word and store it in dictionaries
Begining_word_code = 1
token_pad = 0

token_pos  = 2 
dict_enc = {}
dict_dec = {1: 'START'}
for token, vocab_cnt in data_vocabulary.items():
    if vocab_cnt >= min_thres_value: 
        dict_enc[token] = token_pos
        dict_dec[token_pos ] = token
        token_pos += 1

In [ ]:
#Replacing the words which are not present in the dictionary with UNK Tokens
dict_dec[len(dict_enc)+2] = '<UNK>'
dict_enc['<UNK>'] = len(dict_enc)+2

In [ ]:
token_size = token_pos+1
token_size

3171

In [ ]:
#Creating a function that converts the words to vectors
def vectorize(dict_enc, word, size_of_vector=20):
    transformed_word = np.zeros(shape=(len(word), size_of_vector))
    for k in range(len(word)):
        for l in range(min(len(word[k]), size_of_vector)):
            try:
                transformed_word[k][l] = dict_enc[word[k][l]]
            except:
                transformed_word[k][l] = dict_enc['<UNK>']
    return transformed_word

In [ ]:
length_of_input = 20
length_of_output = 20

In [ ]:
#Vectorizing the training data
train_input_enc = vectorize(
    dict_enc, data_train, size_of_vector=length_of_input)
train_output_enc = vectorize(
    dict_enc, data_outcome, size_of_vector=length_of_output)

print('train_input_enc', train_input_enc.shape)
print('train_output_enc', train_output_enc.shape)

train_input_enc (80000, 20)
train_output_enc (80000, 20)


In [ ]:
#Vectorizing the validation data
valid_input_enc = vectorize(
    dict_enc, data_valid_inp, size_of_vector=length_of_input)
valid_output_enc = vectorize(
    dict_enc, validation_output, size_of_vector=length_of_output)

print('valid_input_enc', valid_input_enc.shape)
print('valid_output_enc', valid_output_enc.shape)

valid_input_enc (20000, 20)
valid_output_enc (20000, 20)


In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [ ]:
#input and output sizes of the vectors
length_of_input = 20
length_of_output = 20

input_to_enc = Input(shape=(length_of_input,))
input_to_dec = Input(shape=(length_of_output,))

In [ ]:
#Defining the encoder and decoder layers
from keras.layers import SimpleRNN
enc = Embedding(token_size, 128, input_length=length_of_input, mask_zero=True)(input_to_enc)
enc = LSTM(512, return_sequences=True, unroll=True)(enc)
last_enc = enc[:,-1,:]
print('encoder', enc)
print('encoder_last', last_enc)

dec = Embedding(token_size, 128, input_length=length_of_output, mask_zero=True)(input_to_dec)
dec = LSTM(512, return_sequences=True, unroll=True)(dec, initial_state=[last_enc, last_enc])
print('decoder', dec)

encoder Tensor("lstm_1/transpose_2:0", shape=(None, 20, 512), dtype=float32)
encoder_last Tensor("strided_slice:0", shape=(None, 512), dtype=float32)
decoder Tensor("lstm_2/transpose_2:0", shape=(None, 20, 512), dtype=float32)


In [ ]:
#Defining the attention layer
from keras.layers import Activation, dot, concatenate
atten = dot([dec, enc], axes=[2, 2])
atten = Activation('softmax', name='attention')(atten)
print('attention', atten)

cxt = dot([atten, enc], axes=[2,1])
print('context', cxt)

combined_cxt_dec = concatenate([cxt, dec])
print('combined_cxt_dec', combined_cxt_dec)

result = TimeDistributed(Dense(512, activation="tanh"))(combined_cxt_dec)
result = TimeDistributed(Dense(token_size, activation="softmax"))(result)
print('result', result)

attention Tensor("attention/truediv:0", shape=(None, 20, 20), dtype=float32)
context Tensor("dot_2/MatMul:0", shape=(None, 20, 512), dtype=float32)
combined_cxt_dec Tensor("concatenate_1/concat:0", shape=(None, 20, 1024), dtype=float32)
result Tensor("time_distributed_2/Reshape_1:0", shape=(None, 20, 3171), dtype=float32)


In [ ]:
#Model summary
Att_model = Model(inputs=[input_to_enc, input_to_dec], outputs=[result])
Att_model.compile(optimizer='adam', loss='binary_crossentropy')
Att_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 128)      405888      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 128)      405888      input_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
training_input_enc = train_input_enc
training_input_dec = np.zeros_like(train_output_enc)
training_input_dec[:, 1:] = train_output_enc[:,:-1]
training_input_dec[:, 0] = Begining_word_code
training_output_dec = np.eye(token_size)[train_output_enc.astype('int')]

validation_input_enc = valid_input_enc
validation_input_dec = np.zeros_like(valid_output_enc)
validation_input_dec[:, 1:] = valid_output_enc[:,:-1]
validation_input_dec[:, 0] = Begining_word_code
validation_output_dec = np.eye(token_size)[valid_output_enc.astype('int')]

In [ ]:
#Running the model
Att_model.fit(x=[training_input_enc, training_input_dec], y=[training_output_dec],
          validation_data=([validation_input_enc, validation_input_dec], [validation_output_dec]),
          batch_size=64, epochs=100)

Att_model.save('Att_model_attention_100k.h5')

/usr/local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 155s 2ms/step - loss: 9.0528e-04 - val_loss: 8.2555e-04
Epoch 2/100
80000/80000 [==============================] - 138s 2ms/step - loss: 7.9726e-04 - val_loss: 7.6897e-04
Epoch 3/100
80000/80000 [==============================] - 135s 2ms/step - loss: 7.5948e-04 - val_loss: 7.4954e-04
Epoch 4/100
80000/80000 [==============================] - 134s 2ms/step - loss: 7.4162e-04 - val_loss: 7.3780e-04
Epoch 5/100
80000/80000 [==============================] - 139s 2ms/step - loss: 7.2903e-04 - val_loss: 7.2872e-04
Epoch 6/100
80000/80000 [==============================] - 139s 2ms/step - loss: 7.1947e-04 - val_loss: 7.2291e-04
Epoch 7/100
80000/80000 [==============================] - 134s 2ms/step - loss: 7.1111e-04 - val_loss: 7.1771e-04
Epoch 8/100
80000/80000 [==============================] - 138s 2ms/step - loss: 7.0362e-04 - val_loss: 7.1391e-04
Epoch 9/100
80000/80000 [=====

/usr/local/lib/python3.6/site-packages/keras/engine/network.py:896: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'strided_slice:0' shape=(None, 512) dtype=float32>, <tf.Tensor 'strided_slice:0' shape=(None, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [ ]:
#Defining the prediction and decoding functions
def to_predict(input_text):
    cleaning_input = clean_sentence(input_text)
    token = [nltk.word_tokenize(cleaning_input)]
    token = [token[0][::-1]]  #reverseing input seq
    inp_enc = vectorize(dict_enc, token, 20)
    inp_dec = np.zeros(shape=(len(inp_enc), length_of_output))
    inp_dec[:,0] = Begining_word_code
    for i in range(1, length_of_output):
        output = Att_model.predict([inp_enc, inp_dec]).argmax(axis=2)
        inp_dec[:,i] = output[:,i]
    return output

def decoding(dict_dec, vector):
    output_text = ''
    for k in vector:
        if k == 0:
            break
        output_text += ' '
        output_text += dict_dec[k]
    return output_text

In [ ]:
#Generating answers for random questions
for i in range(20):
    index_sequence = np.random.randint(1, len(conv_1_short))
    out = to_predict(conv_1_short[index_sequence])
    print ('Question:', conv_1_short[index_sequence])
    print ('Answer:', decoding(dict_dec, out[0]))

Question: how long does this go on
Answer:  i do not know eight or nine
Question: please hurry
Answer:  i will do anything to help you
Question: here in vienna
Answer:  all right send for one hour
Question: not quite you have not announced our engagement yet
Answer:  it must have been <UNK>
Question: cut it out pa he'p al with the truck do not fret honey you goin' to be awright
Answer:  did he always love to you
Question: the first match i got in mind for you is in louisville kentucky
Answer:  you do not <UNK> driving orders do you ever
Question: i just went there for a job
Answer:  then how can we <UNK> this boat
Question: we can monitor the cloud's approach and observe the tests from here
Answer:  is it safe
Question: your lover
Answer:  yes i am here
Question: the tide came in the tide went out i survived that is the headline i survived
Answer:  do not <UNK> me with the details you know how i hate that
Question: hmm primitive ventilation
Answer:  what exactly
Question: have now
Answ

### Below "Q" is the question, "A" is the response/answer from the chatbot and "Orig A" is the actual/Original answer for the question

In [ ]:
#Calculating the Bleu Score of each predicted answer.

from nltk.translate.bleu_score import sentence_bleu

for i in range(20):
    index_sequence = np.random.randint(1, len(conv_1_short))
    out = to_predict(conv_1_short[index_sequence])
    print ('Q:', conv_1_short[index_sequence])
    print ('A:', decoding(dict_dec, out[0]))
    print ('Orig A:', conv_2_short[index_sequence])
    print("BLEU score: ",sentence_bleu([conv_2_short[index_sequence].split()],decoding(dict_dec, out[0]).split()))


Q: i want you to ride with me somethin' does not sound right
A:  i <UNK> your ass with <UNK> <UNK>
Orig A: okay lemme clear it with
BLEU score:  1.1200407237786664e-231
Q: poor larry obedient as ever
A:  keep your voice down
Orig A: keep your voice down
BLEU score:  1.0


/usr/local/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

Q: you sing and dance
A:  a little i just <UNK> from <UNK> film school <UNK> <UNK>
Orig A: a little i just graduated from nyu film school cinema studies
BLEU score:  0.28997844147152074
Q: are you wet
A:  just a little wait a minute woman
Orig A: just a little wait a minute woman
BLEU score:  1.0
Q: are you okay
A:  yes you
Orig A: yes you
BLEU score:  1.491668146240062e-154
Q: he is very realistic
A:  is not the <UNK> <UNK>
Orig A: is not the helmet magnificent
BLEU score:  6.86809206056511e-78
Q: why your mother not breast feed you or something
A:  are you asking me if i have something to prove
Orig A: are you asking me if i have something to prove
BLEU score:  1.0
Q: why are ya goin' so early
A:  <UNK> i get <UNK> you have <UNK>
Orig A: 'cause that is how long it's gonna take baby
BLEU score:  0
Q: it's been months it's not like him
A:  something 's wrong i know it i have heard <UNK> of <UNK> <UNK> south from <UNK>
Orig A: something's wrong i know it i have heard rumors of cholera s

### First sentence in the output is the question and next sentence is the response from the chatbot to the question.

In [ ]:
#Manual Input of Question. 
#First sentence in the output is the question and next sentence is the response from the chatbot
input_text = input()
out = to_predict(input_text)
print (decoding(dict_dec, out[0]))

 How are you?


 fine i am fine how are you


In [ ]:
input_text = input()
out = to_predict(input_text)
print (decoding(dict_dec, out[0]))

 I am good thank you!


 look you are the same


In [ ]:
input_text = input()
out = to_predict(input_text)
print (decoding(dict_dec, out[0]))


 Ofcourse I am the same!


 you are not


In [ ]:
input_text = input()
out = to_predict(input_text)
print (decoding(dict_dec, out[0]))


 why not?


 because i want to


In [ ]:
input_text = input()
out = to_predict(input_text)
print (decoding(dict_dec, out[0]))

 what you want to?


 if i am not being paid to seduce him but he would not kill me you know he did did


In [ ]:
input_text = input()
out = to_predict(input_text)
print (decoding(dict_dec, out[0]))

 what are you even talking about?


 he was shooting at least she was not <UNK>
